## Setup for Log Files

In [1]:
from os.path import exists
from pathlib import Path
import os

home = str(Path.home())
dasklogs = f"{home}/dask-test-logs"
if not exists(dasklogs):
    os.mkdir(dasklogs)

## Initializing the Slurm Cluster

In [3]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    cores=4, 
    memory="8GB",
    processes=2,
    queue="normal",
    shebang='#!/usr/bin/env bash',
    local_directory='/tmp',
    death_timeout="15s",
    interface="ib0",
    log_directory=dasklogs,
    project="boc")

client = Client(cluster)
client

/home/asd/stha/.local/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37426 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: SLURMCluster
Dashboard: http://10.55.50.6:37426/status,
Dashboard: http://10.55.50.6:37426/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.55.50.6:33057,Workers: 0
Dashboard: http://10.55.50.6:37426/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Slurm Job Script

In [4]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /home/asd/stha/dask-test-logs/dask-worker-%J.err
#SBATCH -o /home/asd/stha/dask-test-logs/dask-worker-%J.out
#SBATCH -p normal
#SBATCH -A boc
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=8G
#SBATCH -t 00:30:00

/usr/local/tools/anaconda3/2021.05/bin/python -m distributed.cli.dask_worker tcp://10.55.50.6:33057 --nthreads 2 --nprocs 2 --memory-limit 3.73GiB --name dummy-name --nanny --death-timeout 15s --local-directory /tmp --interface ib0 --protocol tcp://



## Scaling the cluster to 1 node

In [5]:
cluster.scale(2)

## Reading the data into the dataframe

In [6]:
import dask
import dask.dataframe as dd

# dataframe for hpcmod
df = dd.read_csv('data/hpcmod.csv', parse_dates=["_source.Date"],dtype={'_score':'float64', '_source.time':'float64'})  # dtypes specified to read csv properly
# initializing the csv
df

,_index,_type,_id,_score,_source.Date,_source.host,_source.module,_source.path,_source.time,_source.user
npartitions=1,,,,,,,,,,
,object,object,object,float64,datetime64[ns],object,object,object,float64,object
,...,...,...,...,...,...,...,...,...,...


In [7]:
# Shows the beginning of the dataframe
df.head()

,_index,_type,_id,_score,_source.Date,_source.host,_source.module,_source.path,_source.time,_source.user
0,hpcmod,hpc,0CrMz3cBcBNwXO6mF8LV,2.0,2021-01-06 11:23:00,n0,vscode/1.25.1,/usr/local/tools/modulefiles/vscode/1.25.1,1.609950e+09,ackm
1,hpcmod,hpc,0SrMz3cBcBNwXO6mF8LV,2.0,2021-01-06 11:23:00,n0,cuda/9.2,/usr/local/tools/modulefiles/cuda/9.2,1.609950e+09,ackm
2,hpcmod,hpc,0irMz3cBcBNwXO6mF8LV,2.0,2021-01-06 11:23:00,n0,anaconda3/2020.02,/usr/local/tools/modulefiles/anaconda3/2020.02,1.609950e+09,ackm
3,hpcmod,hpc,0yrMz3cBcBNwXO6mF8LV,2.0,2021-01-06 11:23:00,n0,intelmpi/2017/u1,/usr/local/tools/modulefiles/intelmpi/2017/u1,1.609950e+09,ackm
4,hpcmod,hpc,1CrMz3cBcBNwXO6mF8LV,2.0,2021-01-06 11:23:00,n0,R/3.5.1,/usr/local/tools/modulefiles/R/3.5.1,1.609950e+09,ackm


## Dataframe Computations with the Slurm Cluster
### Calculates the length of the dataframe

In [8]:
dflen = len(df)

In [9]:
%%time

# Shows the length of the dataframe
print("Length of dataframe:  " + str(dflen))

Length of dataframe:  10000
CPU times: user 71 µs, sys: 0 ns, total: 71 µs
Wall time: 61.3 µs


In [10]:
client

<Client: 'tcp://10.55.50.6:33057' processes=2 threads=4, memory=7.46 GiB>

### Calculates the number of users that use each module

In [11]:
# Counts the number of users that use each module
count_user = df.groupby("_source.module")['_source.user'].count()

In [12]:
%%time
count_user.compute()

CPU times: user 25.4 ms, sys: 1.56 ms, total: 26.9 ms
Wall time: 522 ms


_source.module
IRIS/20151016_AEBS              182
R/3.4.0                         142
R/3.5.1                          70
R/3.6.3                         122
R/4.0.0                          44
anaconda2/4.3.1                  96
anaconda3/2019.07                48
anaconda3/2020.02               168
anaconda3/2020.11                 2
anaconda3/5.0.1                  27
atom/1.19.4                      95
cuda/10.2                         2
cuda/11.0                         2
cuda/8.0                         10
cuda/9.2                        410
do-not-load/2020                  1
do-not-load/20200811              1
do-not-load/4.6.1                 1
do-not-load/98                   11
do-not-load/julia-1.5.3-mkl       7
dynare/4.5.4                     26
dynareOBC/3.30.53.1962            6
fame/11r5                       183
firefox/62.0.3                    1
firefox/76.0.1                   72
gcc/10.1.0                      183
gcc/7.1.0                       128
gcc/9.3      

In [13]:
client

<Client: 'tcp://10.55.50.6:33057' processes=2 threads=4, memory=7.46 GiB>

### Calculates the number of modules that each user uses

In [15]:
# Counts the number of modules each user uses
count_mod = df.groupby('_source.user')['_source.module'].count()

In [16]:
%%time
count_mod.compute()

CPU times: user 17.2 ms, sys: 571 µs, total: 17.8 ms
Wall time: 178 ms


_source.user
ackm    1401
allj       2
asmo       6
bahs       4
balv       4
brak      17
brys       5
cedw     755
chbo       1
chex       1
cken       1
cori       2
dech     238
gvod       7
hoso     590
kais       6
kart     238
kolm      66
kump       4
manu     368
ninz       4
pavc       3
petb       1
pugt     756
raie      44
ralc       3
reza      93
scec    3788
schl       6
shch       3
shli       3
shrp      58
shxi      21
smfe     152
soyl      12
suyi     112
tery      15
tuzc       6
utan     927
valg       7
wagj      24
walc       4
webl     134
witm       1
xuya      92
ziva      15
Name: _source.module, dtype: int64

In [17]:
client

<Client: 'tcp://10.55.50.6:33057' processes=2 threads=4, memory=7.46 GiB>

## Scaling the cluster to 8 nodes

In [19]:
cluster.scale(16)

### Caclulating users per module with 8 nodes

In [20]:
%%time
count_user.compute()

CPU times: user 29.5 ms, sys: 3.43 ms, total: 32.9 ms
Wall time: 177 ms


_source.module
IRIS/20151016_AEBS              182
R/3.4.0                         142
R/3.5.1                          70
R/3.6.3                         122
R/4.0.0                          44
anaconda2/4.3.1                  96
anaconda3/2019.07                48
anaconda3/2020.02               168
anaconda3/2020.11                 2
anaconda3/5.0.1                  27
atom/1.19.4                      95
cuda/10.2                         2
cuda/11.0                         2
cuda/8.0                         10
cuda/9.2                        410
do-not-load/2020                  1
do-not-load/20200811              1
do-not-load/4.6.1                 1
do-not-load/98                   11
do-not-load/julia-1.5.3-mkl       7
dynare/4.5.4                     26
dynareOBC/3.30.53.1962            6
fame/11r5                       183
firefox/62.0.3                    1
firefox/76.0.1                   72
gcc/10.1.0                      183
gcc/7.1.0                       128
gcc/9.3      

In [21]:
client

<Client: 'tcp://10.55.50.6:33057' processes=16 threads=32, memory=59.68 GiB>